In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint 

In [18]:
#


# create rectangular coordinates
bin_size = 10 # degree increments
start_lat = -90
end_lat = 90
start_long = -180
end_long = 180

min_lats = np.arange(start_lat, end_lat, bin_size)
max_lats = np.arange(start_lat + bin_size, end_lat + bin_size, bin_size)
min_longs = np.arange(start_long, end_long, bin_size)
max_longs = np.arange(start_long + bin_size, end_long + bin_size, bin_size)

# create rectangular limits array with subarrays with elements [minimum_latitude, maximum_latitude, minimum_longitude, maximum_longitude]
# bins = [[min_lats[i], max_lats[i], min_longs[i], max_longs[i]] for i in range(len(end_lats))]
bins = {}
for i in range(len(min_lats)):
    for j in range(len(min_longs)):
        bins[f"{i}_{j}"] = {
            "min_lat": min_lats[i],
            "max_lat": max_lats[i],
            "min_lon": min_longs[j],
            "max_lon": max_longs[j]
        }


pprint(bins)


{'0_0': {'max_lat': -80, 'max_lon': -170, 'min_lat': -90, 'min_lon': -180},
 '0_1': {'max_lat': -80, 'max_lon': -160, 'min_lat': -90, 'min_lon': -170},
 '0_10': {'max_lat': -80, 'max_lon': -70, 'min_lat': -90, 'min_lon': -80},
 '0_11': {'max_lat': -80, 'max_lon': -60, 'min_lat': -90, 'min_lon': -70},
 '0_12': {'max_lat': -80, 'max_lon': -50, 'min_lat': -90, 'min_lon': -60},
 '0_13': {'max_lat': -80, 'max_lon': -40, 'min_lat': -90, 'min_lon': -50},
 '0_14': {'max_lat': -80, 'max_lon': -30, 'min_lat': -90, 'min_lon': -40},
 '0_15': {'max_lat': -80, 'max_lon': -20, 'min_lat': -90, 'min_lon': -30},
 '0_16': {'max_lat': -80, 'max_lon': -10, 'min_lat': -90, 'min_lon': -20},
 '0_17': {'max_lat': -80, 'max_lon': 0, 'min_lat': -90, 'min_lon': -10},
 '0_18': {'max_lat': -80, 'max_lon': 10, 'min_lat': -90, 'min_lon': 0},
 '0_19': {'max_lat': -80, 'max_lon': 20, 'min_lat': -90, 'min_lon': 10},
 '0_2': {'max_lat': -80, 'max_lon': -150, 'min_lat': -90, 'min_lon': -160},
 '0_20': {'max_lat': -80, 'ma

In [29]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

# filters for query
min_magnitude = 5
max_depth = 50          # maximum depth of earthquake
num_results = 2
order_by = 'magnitude'



# setup base query
query = {
    "format":       "geojson",
    "starttime":     "20231201",     # Beginning of 2023 ** testing is 1 month
    "endtime":      "20231231",
    "minmagnitude": min_magnitude,
    "maxdepth":     max_depth,
    "limit":        num_results,
    "orderby":      order_by

}

earthquakes_2023 = {
    "bin_id": [],
    "bin_centre_lat": [],
    "bin_centre_lon": [],
    "quake_lat": [],
    "quake_lon": [],
    "country": [],
    "magnitude": [],
    "depth": [],
    "date": []
    
}

for bin_index, bounds in bins.items():
    query["minlatitude"] = bounds["min_lat"]
    query["maxlatitude"] = bounds["max_lat"]
    query["minlongitude"] = bounds["min_lon"]
    query["maxlongitude"] = bounds["max_lon"]

    # pprint(query)
    response = requests.get(url=base_url, params=query)
    
    features = response.json()["features"]
    metadata = response.json()["metadata"]

    n_earthquakes = metadata["count"]

    
    earthquakes = n_earthquakes > 0

    if earthquakes:
        for quake in features:
            coords = quake["geometry"]["coordinates"]
            earthquakes_2023["quake_lat"].append(coords[1])
            earthquakes_2023["quake_lon"].append(coords[0])
            print(response.url)


    earthquakes_2023["bin_id"].append(bin_index)
    earthquakes_2023["bin_centre_lat"].append(bounds["min_lat"] + bin_size/2)
    earthquakes_2023["bin_centre_lon"].append(bounds["min_lon"] + bin_size/2)


pprint(earthquakes_2023)





# response = requests.get(url)
# response


https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=20231201&endtime=20231231&minmagnitude=5&maxdepth=50&limit=2&orderby=magnitude&minlatitude=-70&maxlatitude=-60&minlongitude=-180&maxlongitude=-170
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=20231201&endtime=20231231&minmagnitude=5&maxdepth=50&limit=2&orderby=magnitude&minlatitude=-70&maxlatitude=-60&minlongitude=-180&maxlongitude=-170
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=20231201&endtime=20231231&minmagnitude=5&maxdepth=50&limit=2&orderby=magnitude&minlatitude=-70&maxlatitude=-60&minlongitude=-180&maxlongitude=-170
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=20231201&endtime=20231231&minmagnitude=5&maxdepth=50&limit=2&orderby=magnitude&minlatitude=-70&maxlatitude=-60&minlongitude=-180&maxlongitude=-170
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=20231201&endtime=20231231&minmagnitude=5&m

KeyboardInterrupt: 